In [ ]:
# ============================================================
# CLASIFICACIÓN MULTICLASE
# Entradas:
#   1) artifacts_preprocesamiento.zip   (contiene las tablas procesadas)
# Salida:
#   resultados.zip (historial de entrenamiento, metadatos y modelo final)
# ============================================================

# ============================================================
# Diseñador de red para clasificación multiclase basado en heurísticas
# + regularización (L2, dropout por capa, early stopping)
# ============================================================

import math
from dataclasses import dataclass
from typing import List


@dataclass
class DisenoRedMulticlase:
    capas: List[int]
    P: int
    rho: float
    l2: float
    dropouts: List[float]
    patience: int
    min_delta: float
    max_epochs: int


def clip(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))


def estimar_parametros_multiclase(n0: int, capas: List[int], K: int) -> int:
    """
    Cuenta parámetros de una red densa:
      - incluye sesgos en cada capa
      - incluye capa de salida (K neuronas)
    """
    if not capas:
        return 0

    P = (n0 + 1) * capas[0]
    for i in range(1, len(capas)):
        P += (capas[i - 1] + 1) * capas[i]
    P += (capas[-1] + 1) * K
    return int(P)


def disenar_red_multiclase(
    d: int,
    n0: int,
    K: int,
    *,
    k: int = 10,
    c1: float = 2.0,
    r: float = 0.5,
    n_min: int = 8,
    L_max: int = 4,
) -> DisenoRedMulticlase:
    """
    Heurística análoga a la binaria:
      - d  : número de muestras (train)
      - n0 : número de variables de entrada
      - K  : número de clases (>= 3)
    """

    # 0. Tope adaptativo de ancho (según tamaño de muestra)
    n_max = min(1024, max(64, math.floor(0.25 * d)))

    # 1. Verificación mínima de viabilidad
    if d < 2 * n0:
        raise ValueError("Dataset muy pequeño: alto riesgo de sobreajuste")

    if K < 3:
        raise ValueError("K debe ser >= 3 para multiclase")

    # 2. Presupuesto total de parámetros
    P_max = math.floor(k * d)

    # 3. Primera capa oculta
    n1_cap_presupuesto = math.floor(P_max / (n0 + 1))
    n1 = min(math.floor(c1 * n0), n1_cap_presupuesto, n_max)

    if n1 < n_min:
        raise ValueError("Presupuesto insuficiente: no se puede ni una capa >= n_min")

    capas = [int(n1)]

    # 4. Construcción iterativa (embudo)
    while True:
        if len(capas) >= L_max:
            break

        n_prev = capas[-1]
        n_new = math.floor(r * n_prev)

        if n_new < n_min:
            break

        n_new = min(n_new, n_max)
        capas.append(int(n_new))

    # 5. Parámetros totales
    P = estimar_parametros_multiclase(n0, capas, K)

    # 6. Recorte si excede presupuesto
    while P > P_max:
        if len(capas) > 1:
            capas.pop()
        else:
            n_old = capas[0]
            capas[0] = math.floor(0.9 * capas[0])
            if capas[0] >= n_old:
                capas[0] = n_old - 1
            if capas[0] < n_min:
                raise ValueError("Presupuesto insuficiente: no cabe una capa >= n_min")

        P = estimar_parametros_multiclase(n0, capas, K)

    # ========================================================
    # 7) Regularización adaptativa
    # ========================================================

    rho = P / P_max if P_max > 0 else 1.0

    # Dropout base por tamaño
    if d < 2000:
        drop_base = 0.35
    elif d < 20000:
        drop_base = 0.25
    else:
        drop_base = 0.15

    # Ajuste por rho
    if rho >= 0.8:
        drop = drop_base + 0.10
    elif rho >= 0.4:
        drop = drop_base
    else:
        drop = drop_base - 0.10
    drop = clip(drop, 0.05, 0.50)

    # Dropout por capa (más al inicio)
    dropouts: List[float] = []
    for i in range(1, len(capas) + 1):
        di = drop * (1.0 - 0.15 * (i - 1))
        di = clip(di, 0.05, 0.50)
        dropouts.append(float(di))

    # L2 base por tamaño
    if d < 2000:
        l2_base = 1e-3
    elif d < 20000:
        l2_base = 3e-4
    else:
        l2_base = 1e-4

    # Ajuste por rho
    if rho >= 0.8:
        l2 = 3.0 * l2_base
    elif rho >= 0.4:
        l2 = 1.0 * l2_base
    else:
        l2 = 0.3 * l2_base
    l2 = clip(l2, 1e-6, 3e-3)

    # Early stopping
    if d < 2000:
        patience = 20
        max_epochs = 400
    elif d < 20000:
        patience = 15
        max_epochs = 200
    else:
        patience = 10
        max_epochs = 100

    min_delta = 1e-4

    return DisenoRedMulticlase(
        capas=capas,
        P=int(P),
        rho=float(rho),
        l2=float(l2),
        dropouts=dropouts,
        patience=int(patience),
        min_delta=float(min_delta),
        max_epochs=int(max_epochs),
    )

In [ ]:
import pandas as pd
import numpy as np
import zipfile

import tensorflow as tf
from tensorflow import keras

# Reproducibilidad (opcional)
SEED = 7
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ============================================================
# 1) Abrir ZIP y leer train/val/test
# ============================================================

ZIP_PATH = "artifacts_preprocesamiento.zip"

def read_csv_from_zip(zip_path: str, csv_name: str) -> pd.DataFrame:
    with zipfile.ZipFile(zip_path, "r") as z:
        with z.open(csv_name) as f:
            return pd.read_csv(f)

train = read_csv_from_zip(ZIP_PATH, "train_final.csv")
val   = read_csv_from_zip(ZIP_PATH, "val_final.csv")
test  = read_csv_from_zip(ZIP_PATH, "test_final.csv")

# ============================================================
# 2) Definir target y armar X/y
# ============================================================

TARGET_COL = "target"

X_train = train.drop(columns=[TARGET_COL])
y_train_raw = train[TARGET_COL].astype(int)

X_val = val.drop(columns=[TARGET_COL])
y_val_raw = val[TARGET_COL].astype(int)

X_test = test.drop(columns=[TARGET_COL])
y_test_raw = test[TARGET_COL].astype(int)

d  = X_train.shape[0]
n0 = X_train.shape[1]

# ============================================================
# 3) Mapeo automático de etiquetas (clase_original -> idx 0..K-1)
# ============================================================

classes_original = np.sort(y_train_raw.unique())
K = int(len(classes_original))

class_to_index = {int(c): int(i) for i, c in enumerate(classes_original)}
index_to_class = {int(i): int(c) for i, c in enumerate(classes_original)}

def map_labels(y_series: pd.Series, mapping: dict) -> np.ndarray:
    y_mapped = y_series.map(mapping)
    if y_mapped.isna().any():
        unseen = sorted(set(y_series.unique()) - set(mapping.keys()))
        raise ValueError(
            f"Clases en val/test no vistas en train: {unseen}. "
            f"Revisa el split/stratify o la construcción de datasets."
        )
    return y_mapped.astype(int).values

y_train = map_labels(y_train_raw, class_to_index)
y_val   = map_labels(y_val_raw, class_to_index)
y_test  = map_labels(y_test_raw, class_to_index)

print("d =", d)
print("n0 =", n0)
print("K =", K)
print("classes_original (train) =", classes_original.tolist())
print("class_to_index =", class_to_index)

# ============================================================
# 4) Detección automática de desbalanceo + class_weight
# ============================================================

counts = pd.Series(y_train).value_counts().sort_index()
counts = counts.reindex(range(K), fill_value=0)

min_c = int(counts.min())
max_c = int(counts.max())
imbalance_ratio = (max_c / min_c) if min_c > 0 else float("inf")

IMBALANCE_THRESHOLD = 1.5
usa_class_weight = bool(imbalance_ratio >= IMBALANCE_THRESHOLD)

class_weight = None
if usa_class_weight:
    total = float(len(y_train))
    class_weight = {int(i): float(total / (K * counts.loc[i])) for i in range(K)}

print("\n=== Desbalanceo (train, índices internos) ===")
print("counts (idx->n):\n", counts.to_string())
print("imbalance_ratio (max/min):", float(imbalance_ratio))
print("IMBALANCE_THRESHOLD:", float(IMBALANCE_THRESHOLD))
print("¿Se usará class_weight?:", usa_class_weight)
if usa_class_weight:
    print("class_weight (idx->w):", class_weight)

In [ ]:
# ============================================================
# 3) Configuración de evaluación (simetría vs clases críticas)
# ============================================================
#
# True  -> clases "simétricas" -> optimizamos Accuracy
# False -> robustez ante desbalanceo -> optimizamos Macro-F1
#
# NOTA: En cualquier caso reportamos Accuracy, Macro-F1 y Weighted-F1.
# ============================================================

CLASES_SIMETRICAS = False

In [ ]:
diseno = disenar_red_multiclase(d, n0, K)
print(diseno)

In [ ]:
capas     = diseno.capas
l2_value  = diseno.l2
dropouts  = diseno.dropouts
patience  = diseno.patience
min_delta = diseno.min_delta
max_epochs= diseno.max_epochs

def build_multiclass_mlp(n0: int, K: int, capas: list, l2_value: float, dropouts: list) -> keras.Model:
    assert len(capas) == len(dropouts), "capas y dropouts deben tener la misma longitud"

    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(n0,)))

    for units, dr in zip(capas, dropouts):
        model.add(
            keras.layers.Dense(
                units,
                activation="relu",
                kernel_regularizer=keras.regularizers.l2(l2_value)
            )
        )
        model.add(keras.layers.Dropout(dr))

    model.add(keras.layers.Dense(K, activation="softmax"))
    return model

model = build_multiclass_mlp(n0=n0, K=K, capas=capas, l2_value=l2_value, dropouts=dropouts)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
    ],
)

model.summary()

In [ ]:
# ============================================================
# 5) Entrenar + validar (EarlyStopping) + class_weight si aplica
# ============================================================

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=patience,
        min_delta=min_delta,
        restore_best_weights=True,
        verbose=1
    )
]

BATCH_SIZE = 32

fit_kwargs = {}
if usa_class_weight and (class_weight is not None):
    fit_kwargs["class_weight"] = class_weight  # SOLO si hay desbalanceo

history = model.fit(
    X_train.astype(np.float32),
    y_train,
    validation_data=(X_val.astype(np.float32), y_val),
    epochs=max_epochs,
    batch_size=BATCH_SIZE,
    verbose=1,
    callbacks=callbacks,
    **fit_kwargs
)

In [ ]:
# ============================================================
# 6) Evaluación + métricas robustas + ZIP final
# ============================================================

import os
import json
import zipfile
import pandas as pd
import numpy as np

def confusion_matrix_multiclass(y_true: np.ndarray, y_pred: np.ndarray, K: int) -> np.ndarray:
    cm = np.zeros((K, K), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[int(t), int(p)] += 1
    return cm

def accuracy_from_cm(cm: np.ndarray) -> float:
    total = cm.sum()
    return float(np.trace(cm) / total) if total else 0.0

def f1_per_class_from_cm(cm: np.ndarray):
    K = cm.shape[0]
    f1s = np.zeros(K, dtype=float)
    support = cm.sum(axis=1).astype(float)
    for k in range(K):
        TP = cm[k, k]
        FP = cm[:, k].sum() - TP
        FN = cm[k, :].sum() - TP
        denom = 2*TP + FP + FN
        f1s[k] = (2*TP / denom) if denom else 0.0
    return f1s, support

def f1_macro_from_cm(cm: np.ndarray) -> float:
    f1s, _ = f1_per_class_from_cm(cm)
    return float(np.mean(f1s)) if len(f1s) else 0.0

def f1_weighted_from_cm(cm: np.ndarray) -> float:
    f1s, support = f1_per_class_from_cm(cm)
    total = float(support.sum())
    if total <= 0:
        return 0.0
    w = support / total
    return float((w * f1s).sum())

def print_confusion_matrix_multiclass(cm: np.ndarray, *, title="Matriz de confusión"):
    print(f"\n{title}: (filas=True, columnas=Pred) [índices internos 0..K-1]")
    K = cm.shape[0]
    header = "True\\Pred | " + " ".join([f"{j:6d}" for j in range(K)])
    print(header)
    print("-" * len(header))
    for i in range(K):
        row = " ".join([f"{cm[i,j]:6d}" for j in range(K)])
        print(f"{i:9d} | {row}")

# ---- VALIDACIÓN ----
p_val = model.predict(X_val.astype(np.float32), verbose=0)
yhat_val = np.argmax(p_val, axis=1).astype(int)

cm_val = confusion_matrix_multiclass(y_val, yhat_val, K)
acc_val = accuracy_from_cm(cm_val)
f1m_val = f1_macro_from_cm(cm_val)
f1w_val = f1_weighted_from_cm(cm_val)

metric_principal = "Accuracy" if CLASES_SIMETRICAS else "Macro-F1"
score_val_principal = acc_val if CLASES_SIMETRICAS else f1m_val

print("\n=== Configuración de decisión (multiclase) ===")
print("Métrica principal:", metric_principal)
print("¿Clases 'simétricas'?:", bool(CLASES_SIMETRICAS))
print("Regla de decisión:", "argmax(softmax)")
print("Score en VALIDACIÓN (métrica principal):", float(score_val_principal))

print("¿Se usó class_weight?:", bool(usa_class_weight))
if usa_class_weight:
    print("IMBALANCE_THRESHOLD:", float(IMBALANCE_THRESHOLD))
    print("imbalance_ratio (train max/min):", float(imbalance_ratio))

print_confusion_matrix_multiclass(cm_val, title="Matriz de confusión (VALIDACIÓN)")

print("\nMétricas en validación:")
print("-----------------------------------")
print(f"{'Métrica':<15} | {'Valor':>10}")
print("-----------------------------------")
print(f"{'Accuracy':<15} | {acc_val:10.4f}")
print(f"{'Macro-F1':<15} | {f1m_val:10.4f}")
print(f"{'Weighted-F1':<15} | {f1w_val:10.4f}")
print("-----------------------------------")

# ---- TEST ----
p_test = model.predict(X_test.astype(np.float32), verbose=0)
yhat_test = np.argmax(p_test, axis=1).astype(int)

cm_test = confusion_matrix_multiclass(y_test, yhat_test, K)
acc_test = accuracy_from_cm(cm_test)
f1m_test = f1_macro_from_cm(cm_test)
f1w_test = f1_weighted_from_cm(cm_test)

print_confusion_matrix_multiclass(cm_test, title="Matriz de confusión (TEST)")

print("\nMétricas en test:")
print("-----------------------------------")
print(f"{'Métrica':<15} | {'Valor':>10}")
print("-----------------------------------")
print(f"{'Accuracy':<15} | {acc_test:10.4f}")
print(f"{'Macro-F1':<15} | {f1m_test:10.4f}")
print(f"{'Weighted-F1':<15} | {f1w_test:10.4f}")
print("-----------------------------------")

# ============================================================
# 7) Guardar resultados y empaquetar ZIP final
# ============================================================

OUT_DIR = "salida_multiclase"
ZIP_NAME = "resultados.zip"
os.makedirs(OUT_DIR, exist_ok=True)

metadata = {
    "problem_type": "multiclass_classification",

    "n_samples_train": int(d),
    "n_features": int(n0),
    "n_classes": int(K),

    # --- mapeo crítico ---
    "classes_original_train": [int(c) for c in classes_original],
    "class_to_index": {str(int(k)): int(v) for k, v in class_to_index.items()},
    "index_to_class": {str(int(k)): int(v) for k, v in index_to_class.items()},

    "architecture": capas,
    "l2": float(l2_value),
    "dropouts": dropouts,
    "patience": int(patience),
    "min_delta": float(min_delta),
    "max_epochs": int(max_epochs),

    # --- decisión / prioridad ---
    "clases_simetricas": bool(CLASES_SIMETRICAS),
    "metrica_principal": metric_principal,
    "decision_rule": "argmax_softmax",

    # --- desbalanceo / pesos ---
    "imbalance_threshold": float(IMBALANCE_THRESHOLD),
    "imbalance_ratio_train_max_min": float(imbalance_ratio),
    "used_class_weight": bool(usa_class_weight),
    "class_counts_train_indexed": {str(int(k)): int(v) for k, v in counts.items()},
    "class_weight_indexed": None if not usa_class_weight else {str(int(k)): float(v) for k, v in class_weight.items()},

    # --- resultados ---
    "metrics_val": {
        "accuracy": float(acc_val),
        "macro_f1": float(f1m_val),
        "weighted_f1": float(f1w_val),
    },
    "metrics_test": {
        "accuracy": float(acc_test),
        "macro_f1": float(f1m_test),
        "weighted_f1": float(f1w_test),
    },

    "confusion_matrix_val_indexed": cm_val.tolist(),
    "confusion_matrix_test_indexed": cm_test.tolist(),
}

with open(os.path.join(OUT_DIR, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=2)

model_path = os.path.join(OUT_DIR, "modelo.keras")
model.save(model_path)

history_path = os.path.join(OUT_DIR, "historial_entrenamiento.csv")
pd.DataFrame(history.history).to_csv(history_path, index=False)

with zipfile.ZipFile(ZIP_NAME, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file in [model_path, history_path, os.path.join(OUT_DIR, "metadata.json")]:
        zipf.write(file, arcname=os.path.basename(file))

print(f"\n✔ ZIP generado correctamente: {ZIP_NAME}")